# Calculation of bulk properties with molecular dynamics

In this notebook, we will use a molecular dynamics simulation to compute the thermal expansion coefficient of solid Xe at 77 K and 1 atm pressure. In the isothermal-isobaric ensemble (number of atoms, temperature, and pressure are all held constant - the NPT ensemble), the thermal expansion coefficient can be related to the volume, enthalpy, pressure, and temperature as:

$$ \alpha_V = \frac{1}{Vk_BT^2}\Bigl(\langle HV \rangle - \langle H\rangle \langle V \rangle \Bigr) $$

where $k_B$ is Boltzmann's constant and the enthalpy is:

$$ H = E + PV $$

The quantities inside the angle brackets represent average values. So $\langle H \rangle \langle V \rangle$ is the average enthalpy times the average volume, while $\langle HV \rangle$ is the average of the product of $H$ and $V$ for every frame.

To perform this calculation, the `xe-729.pdb` file contains a lattice of 729 Xe atoms, and the `xenon.xml` file contains the Lennard-Jones parameters for the Xe atom. Because we want to simulate bulk Xe, and not the properties of a Xe nanocrystal, we will use a periodic boundary condition with a cutoff of 1.2 nm. Also, to maintain constant temperature, a Langevin thermostat is used with a target temperature of 77 K and a collision rate of 1 ps$^{-1}$. Finally, constant pressure is maintained with a Monte Carlo barostat with an attempt interval set to 25 steps.

In the code below, every 1000 steps output is written to a `teajectory.pdb` file, which can be viewed with VMD or another visualization package. In addition, the step number, time, PE, KE, total energy, temperature, volume, and density are written to a `data.csv` file every 1000 steps and to standard output every 10000 steps.

The simulation is initialized with random velocities drawn from a 77 K distribution, then allowed to minimize and equilibrate before the production run is started. The production run goes for 1000000 steps.

In [ ]:
##########################################################################
# this script was generated by openmm-builder. to customize it further,
# you can save the file to disk and edit it with your favorite editor.
##########################################################################

from __future__ import print_function
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout

pdb = app.PDBFile('xe-729.pdb')
forcefield = app.ForceField('xenon.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.CutoffPeriodic, 
    nonbondedCutoff=1.2*unit.nanometers, constraints=None, rigidWater=False)
integrator = mm.LangevinIntegrator(77*unit.kelvin, 1.0/unit.picoseconds, 
    5.0*unit.femtoseconds)
system.addForce(mm.MonteCarloBarostat(1*unit.atmospheres, 77*unit.kelvin, 25))

platform = mm.Platform.getPlatformByName('CPU')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

print('Minimizing...')
simulation.minimizeEnergy(maxIterations=100)

simulation.context.setVelocitiesToTemperature(77*unit.kelvin)
print('Equilibrating...')
simulation.step(100)

simulation.reporters.append(app.PDBReporter('trajectory.pdb', 1000))
simulation.reporters.append(app.StateDataReporter(stdout, 10000, step=True, 
    time=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, 
    temperature=True, volume=True, density=True, separator='\t'))
simulation.reporters.append(app.StateDataReporter('data.csv', 1000, step=True, 
    time=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, 
    temperature=True, volume=True, density=True, separator=','))

print('Running Production...')
simulation.step(1000000)
print('Done!')

At this point, you can open the `trajectory.pdb` file in VMD to ensure that you see a Xe solid, where the atoms are vibrating about their lattice sites. Under "Extensions > Analysis" you can find the menu for calculating the radial distribution function between atoms: setting `name = Xe1` for both selection fields will compute the function.

Now we can analyze the results of the simulation and compute the thermal expansion coefficient.

In [ ]:
import pandas as pd

df = pd.read_csv('data.csv')
df.describe()

The thermostat does not force the temperature to 77 K at every frame; it works by altering the equation of motion to add random forces to slow particles and friction to fast ones, such that over time, the average temperature approaches 77 K. We can visualize a histogram of the temperature (and indeed the other properties) to see its distribution.

In [ ]:
from matplotlib import pyplot as plt

fig,ax = plt.subplots()
ax.hist(df['Temperature (K)'],bins=100)
ax.set_xlabel('Temperature (K)')
ax.set_ylabel('Frames')

Now we are ready to compute the thermal expansion coefficient. Because the units of pressure in the simulation are atm and the units of volume are nm$^3$, some unit conversions are needed for calculating enthalpy: $PV$ must have the same units as $E$ (kJ/mol). This is accomplished by converting atm to Pa (J/m$^3$), converting atoms to moles with Avogadro's number, converting J to kJ, and nm$^3$ to m$^3$.

Unit conversions are available in the scipy.constants module.

In [ ]:
import scipy.constants as spc

atm = spc.value('standard atmosphere') # 101325
NA = spc.value('Avogadro constant')

In [ ]:
import numpy as np

V = df['Box Volume (nm^3)']
T = df['Temperature (K)']
E = df['Total Energy (kJ/mole)']

Vavg = np.mean(V)
Tavg = np.mean(T)
Eavg = np.mean(E)
P = 1.0
print(Vavg,Tavg,Eavg)

Now we can compute the enthalpy in each frame, the total average enthalpy, the product of enthalpy and volume ($HV$) for each frame, and the average $\langle HV \rangle$.

In [ ]:
H = E + P*V*atm*NA/spc.kilo*(spc.nano**3) #units kJ/mol
Havg = Eavg + P*Vavg*atm*NA/spc.kilo*(spc.nano**3) #units kJ/mol
HVavg = np.mean(H*V) #units kJ nm^3 / mol

And finally, the thermal expansion coefficient $\alpha$:

In [ ]:
print(f'alpha = {1/(Vavg*spc.value("Boltzmann constant")*NA/spc.kilo*Tavg**2)*(HVavg - Havg*Vavg)} 1/K')